In [1]:
# 丙戊酸钠-双相情感分类模型集成

In [3]:
import pandas as pd
import numpy as np

import sys
import re
import os
project_path = os.getcwd()

## load data

In [25]:
df_model =pd.read_excel(project_path +'/data/2_auc_df_model_data_forward_模型集成.xlsx')
if 'Unnamed: 0' in df_model.columns:
    df_model = df_model.drop(['Unnamed: 0'], axis=1)

In [26]:
df_model.shape

(164, 8)

In [27]:
df_model.columns

Index(['target_dosage', 'APD', 'VPA_tdm', 'PLCR', 'PDW', 'indirect_bilirubin',
       'PCV', 'Hb'],
      dtype='object')

In [28]:
df_model['target_dosage'].value_counts()

1    135
0     29
Name: target_dosage, dtype: int64

In [29]:
df_model.head()

,target_dosage,APD,VPA_tdm,PLCR,PDW,indirect_bilirubin,PCV,Hb
0,1,1,97.3,NaN,NaN,NaN,NaN,NaN
1,1,0,78.2,26.1,15.7,7.8,0.404,132.0
2,1,0,37.4,23.9,15.8,7.5,0.420,138.0
3,0,0,69.2,15.0,15.5,3.5,0.370,121.0
4,1,0,85.1,44.8,17.8,3.5,0.490,156.0


In [30]:
discrete_col=['APD']
continuous_col=[x for x in df_model.columns if x not in discrete_col]
continuous_col.remove('target_dosage')

## 数据归一化

In [31]:
# 防止不同维特征数据差距过大，影响建模效果
max_list=[]
for i in continuous_col:
    max_value = df_model[i].max()
    max_list.append(max_value)
    df_model[i]=df_model[i].apply(lambda x: round(x/max_value,3))

In [32]:
df_model.columns

Index(['target_dosage', 'APD', 'VPA_tdm', 'PLCR', 'PDW', 'indirect_bilirubin',
       'PCV', 'Hb'],
      dtype='object')

In [33]:
df_max_value=pd.DataFrame(data={'features':continuous_col,
                               'max_value':max_list})

In [34]:
df_max_value

,features,max_value
0,VPA_tdm,150.800
1,PLCR,53.400
2,PDW,17.800
3,indirect_bilirubin,15.300
4,PCV,0.495
5,Hb,168.000


## 插补数据

In [35]:
# 使用随机森林对缺失值进行插补
import pandas as pd
pd.set_option('mode.chained_assignment', None)
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
def missing_value_interpolation(df):
    df = df.reset_index(drop=True)
    # 提取存在缺失值的列名
    missing_list = []
    for i in df.columns:
        if df[i].isnull().sum() > 0:
            missing_list.append(i)
    missing_list_copy = missing_list.copy()
    # 用该列未缺失的值训练随机森林，然后用训练好的rf预测缺失值
    for i in range(len(missing_list)):
        name=missing_list[0]
        df_missing = df[missing_list_copy]
        # 将其他列的缺失值用0表示。
        missing_list.remove(name)
        for j in missing_list:
            df_missing[j]=df_missing[j].astype('str').apply(lambda x: 0 if x=='nan' else x)
        df_missing_is = df_missing[df_missing[name].isnull()]
        df_missing_not = df_missing[df_missing[name].notnull()]
        y = df_missing_not[name]
        x = df_missing_not.drop([name],axis=1)

        rfr = RandomForestRegressor(n_estimators=300,
                                    random_state=3)
        rfr.fit(x, y)
        #预测缺失值
        predict = rfr.predict(df_missing_is.drop([name],axis=1))
        #填补缺失值
        df.loc[df[name].isnull(),name] = predict
    return df

In [36]:
# 插补建模数据
df_model_cb=missing_value_interpolation(df_model)

In [37]:
# 保存插补数据
df_model_cb.to_excel(project_path + '/data/df_model_data_插补.xlsx')

## SMOTE

In [38]:
# 进行过采样
from imblearn.over_sampling import SMOTE,ADASYN 
tran_x=df_model_cb.drop(['target_dosage'],axis=1)
tran_y=df_model_cb['target_dosage']
sm = SMOTE(random_state=0)

tran_x_sm,tran_y_sm = sm.fit_resample(tran_x,tran_y)

## model

In [39]:
import xgboost
# XGBoost模型
xgb_model=xgboost.XGBClassifier(max_depth=5,
                        learning_rate=0.1,
                        n_estimators=500,
                        min_child_weight=0.6,
                        eta=0.1,
                        gamma=0.5,
                        reg_lambda=8,
                        subsample=0.5,
                        colsample_bytree=0.9,
                        nthread=4,
                        scale_pos_weight=1,
                        random_state=3)
xgb_model.fit(tran_x_sm,tran_y_sm)

XGBClassifier(colsample_bytree=0.9, eta=0.1, gamma=0.5, max_depth=5,
              min_child_weight=0.6, n_estimators=500, nthread=4, random_state=3,
              reg_lambda=8, subsample=0.5)

## save model

In [40]:
import pickle
pickle.dump(xgb_model,open(project_path+'/data/result/xgb_model.pkl','wb'))

## predict dosage

In [93]:
APD=1
VPA_tdm=50
PLCR=31.4
PDW=16.7
indirect_bilirubin=np.nan
PCV=0.367
Hb=120

In [94]:
# 连续变量归一化处理
VPA_tdm=round(VPA_tdm/150.8,3)
PLCR=round(PLCR/53.4,3)
PDW=round(PDW/17.8,3)
indirect_bilirubin=round(indirect_bilirubin/15.3,3)
PCV=round(PCV/0.495,3)
Hb=round(Hb/168,3)

In [102]:
indirect_bilirubin

nan

In [95]:
df_test=pd.DataFrame(data={'APD':[APD],
                           'VPA_tdm':[VPA_tdm],
                           'PLCR':[PLCR],
                           'PDW':[PDW],
                           'indirect_bilirubin':[indirect_bilirubin],
                           'PCV':[PCV],
                           'Hb':[Hb]})

In [96]:
# 重新加载预测模型
pre_model=pickle.load(open(project_path+'/data/result/xgb_model.pkl','rb'))

In [97]:
pre_model

XGBClassifier(colsample_bytree=0.9, eta=0.1, gamma=0.5, max_depth=5,
              min_child_weight=0.6, missing=nan, n_estimators=500, nthread=4,
              random_state=3, reg_lambda=8, subsample=0.5)

In [98]:
pre=pre_model.predict(df_test)

In [99]:
pre

array([0], dtype=int64)

In [100]:
# 标签转换为日剂量
if pre[0]==0:
    dosage='0.5g'
elif pre[0]==1:
    dosage='1g'

In [101]:
dosage

'0.5g'